# Factorisation LU
## Fiche de correction
### Alban Gossard (gossard@insa-toulouse.fr)

Mettez ci-dessous les imports classiques de librairie Python

In [1]:
%matplotlib inline
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

Le but de ce TP est d'implémenter la factorisation LU, avec et sans permutations, et de tester ces méthodes pour résoudre un système linéaire. Dans cette dernière partie, on implémentera un algorithme de résolution de systèmes triangulaires.

## 1) Algorithme de factorisation LU standard

On se propose tout d'abord d'implémenter un code de factorisation LU qui prendra en entrée une matrice $A$ carré de taille quelconque et qui donnera en sortie le couple de matrice L et U correspondant à la factorisation LU de la matrice A. On rappelle que la factorisation LU s'obtient à partir de la méthode de pivot de Gauss. Plus précisément, la factorisation LU peut se résumer à l'algorithme suivant

$$
A^{(0)} = A\in M_n(\mathbb{R}),\\
L^{(0)} = 0 \in M_n(\mathbb{R}),\\
\textrm{Pour}\; k = 0,1,2,\cdots,n-1:\;
\left\{\begin{array}{ll}
\ell_i^{(k)} = \left\{\begin{array}{ll}
0,\;\textrm{pour}\;0\leq i\leq k-1,\\
\frac{a_{i,k}^{(k)}}{a_{k,k}^{(k)}},\;\textrm{pour}\;k\leq i\leq n-1,
\end{array}\right.\\
L^{(k+1)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_i,\; \textrm{pour}\; j = k\;\textrm{et}\; k\leq i\leq n-1,
\\ L^{(k)}_{i,j}\; \textrm{sinon},
\end{array}\right.\\
B^{(k)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_{i} a^{(k)}_{k,j},\; \textrm{pour}\; k+1\leq i\leq n-1 \;\textrm{et}\; k\leq j\leq n-1,
\\ 0\; \textrm{sinon},
\end{array}\right.,\\
A^{(k+1)} = A^{(k)} - B^{(k)},\\
\end{array}\right.
$$

où
- $\ell^{(k)}$ est un vecteur de $\mathbb{R}^n$,
- $A^{(k)}$ est une matrice de $M_n(\mathbb{R})$ dont on note les coefficients $(a^{(k)}_{i,j})_{0\leq i,j\leq n-1}$.

A la fin de cette algorithme, on obtient $L = L^{(n-1)}$ et $U = A^{(n-1)}$.

> **A faire :** Implémenter la fonction `LU` qui permet de calculer la factorisation LU d'une matrice $A$ carré de dimension quelconque à partir de l'algorithme précédent. On prendra soin de vérifier si le pivot est non-nul et, dans le cas contraire, on renverra une erreur à l'utilisateur. On pourra tester la fonction sur la matrice
$$
A = \left(\begin{array}{ccc}
2 & 1 & 1 \\
6 & 2 & 1 \\
-2 & 2 & 1
\end{array}\right),
$$
dont la décomposition LU est donnée par
$$
L = \left(\begin{array}{ccc}
1 & 0 & 0 \\
3 & 1 & 0 \\
-1 & -3 & 1
\end{array}\right)\quad\textrm{et}\quad U = \left(\begin{array}{ccc}
2 & 1 & 1 \\
0 & -1 & -2 \\
0 & 0 & -4
\end{array}\right).
$$

In [2]:
def LU(A, verbose=False):
    Ak = np.copy(A)
    Lk = np.zeros(A.shape)
    n = A.shape[0]
    for k in range(n):
        if verbose:
            print("Iteration",k)
        lk = np.zeros(n)
        if abs(Ak[k,k])<1e-15:
            raise Exception("Le pivot est nul")
        lk[k:] = Ak[k:,k]/Ak[k,k]
        if verbose:
            print("lk=",lk)
        Lk[k:,k] = lk[k:]
        Bk = np.zeros(A.shape)
        Bk[k+1:,k:] = lk[k+1:].reshape((-1,1))*Ak[k,k:].reshape((1,-1))
        if verbose:
            print("Bk=",Bk)
        Ak = Ak-Bk
        if verbose:
            print("Ak=",Ak,"\n")
    L=Lk
    M=Ak
    return L,M

In [3]:
A = np.array([[2,1,1],[6,2,1],[-2,2,1]])
L,U = LU(A,verbose=True)
print("A=",A)
print("L=",L)
print("U=",U)
print("LU=",L.dot(U))
if np.testing.assert_array_equal(L.dot(U),A) is None:
    print("A=LU")

Iteration 0
lk= [ 1.  3. -1.]
Bk= [[ 0.  0.  0.]
 [ 6.  3.  3.]
 [-2. -1. -1.]]
Ak= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  3.  2.]] 

Iteration 1
lk= [ 0.  1. -3.]
Bk= [[0. 0. 0.]
 [0. 0. 0.]
 [0. 3. 6.]]
Ak= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  0. -4.]] 

Iteration 2
lk= [0. 0. 1.]
Bk= [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Ak= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  0. -4.]] 

A= [[ 2  1  1]
 [ 6  2  1]
 [-2  2  1]]
L= [[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-1. -3.  1.]]
U= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  0. -4.]]
LU= [[ 2.  1.  1.]
 [ 6.  2.  1.]
 [-2.  2.  1.]]
A=LU


## 2) Algorithme de factorisation LU avec stratégie de pivot partiel

On passe maintenant à la factorisation LU avec la stratégie de pivot partiel. Cette méthode met en place, durant le pivot de Gauss, des permutations sur les lignes qui permettent de diminuer les erreurs numériques et de parer au problème d'un pivot nul. Ces permutations sur les lignes vont faire en sorte que le pivot dans la méthode de Gauss est toujours le plus grand possible. En pratique, à chaque étape $k$, lors du choix du pivot, on va chercher dans la colonne $k$ le plus grand pivot (en valeur absolue) puis on va procèder à une permutation entre la ligne de ce pivot et la ligne $k$. Plus précisément, la factorisation LU avec permutations peut se résumer à l'algorithme suivant

$$
A^{(0)} = A\in M_n(\mathbb{R}),\\
L^{(0)} = 0\in M_n(\mathbb{R}),\\
P^{(0)} = \textrm{Id}\in M_n(\mathbb{R}),\\
\textrm{Pour}\; k = 0,1,2,\cdots,n-1:\;
\left\{\begin{array}{ll}
m = \textrm{argmax}_{k\leq j \leq n-1} |a_{j,k}^{(k)}|\\
Q^{k,m} = \textrm{Matrice de permutation des indices $k$ et $m$}\\
P^{(k+1)} = Q^{k,m}P^{(k)},\\
L^{(k+1/2)} = Q^{k,m}L^{(k)},\\
A^{(k+1/2)} = Q^{k,m}A^{(k)},\\
\ell_i^{(k)} = \left\{\begin{array}{ll}
0,\;\textrm{pour}\;0\leq i\leq k-1,\\
\frac{a_{i,k}^{(k+1/2)}}{a_{k,k}^{(k+1/2)}},\;\textrm{pour}\;k\leq i\leq n-1,
\end{array}\right.\\
L^{(k+1)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_i,\; \textrm{pour}\; j = k\;\textrm{et}\; k\leq i\leq n-1,
\\ L^{(k+1/2)}_{i,j}\; \textrm{sinon},
\end{array}\right.\\
B^{(k)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_{i} a^{(k+1/2)}_{k,j},\; \textrm{pour}\; k+1\leq i\leq n-1 \;\textrm{et}\; k\leq j\leq n-1,
\\ 0\; \textrm{sinon},
\end{array}\right.,\\
A^{(k+1)} = A^{(k+1/2)} - B^{(k)},\\
\end{array}\right.
$$

A la fin de cette algorithme, on obtient $L = L^{(n-1)}$, $U = A^{(n-1)}$ et $P = P^{(n-1)}$. La matrice de permutation n'a pas vraiment besoin d'être assemblée, on pourra simplement écrire une fonction `Permut` qui prendra en argument une matrice et le couple d'indice et donnera en sortie la matrice avec ses lignes permutées. Afin d'identifier l'indice du plus grand pivot, on pourra s'aider de la fonction `argmax`. On rappelle de plus que, pour une matrice de permutation $P$, on a

$$
P^{-1} = P^T.
$$

> **A faire :** Implémenter la fonction `PLU` qui permet de calculer la factorisation LU d'une matrice $A$ carré de dimension quelconque à partir de l'algorithme précédent. On prendra soin de vérifier si le pivot est non-nul et, dans le cas contraire, on renverra une erreur à l'utilisateur. On pourra tester la fonction sur la matrice
$$
A = \left(\begin{array}{ccc}
1 & 1 & 1 & 1 \\
1 & 1 & 3 & 3 \\
1 & 1 & 2 & 3\\
1 & 3 & 3 & 3
\end{array}\right),
$$
dont la décomposition PLU est donnée par
$$
P = \left(\begin{array}{ccc}
1 & 0 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0
\end{array}\right),\quad L = \left(\begin{array}{ccc}
1 & 0 & 0 & 0\\
1 & 1 & 0 & 0\\
1 & 0 & 1 & 0\\
1 & 0 & 1/2 & 1
\end{array}\right)\quad\textrm{et}\quad U = \left(\begin{array}{ccc}
1 & 1 & 1 & 1\\
0 & 2 & 2 & 2\\
0 & 0 & 2 & 2\\
0 & 0 & 0 & 1
\end{array}\right).
$$

In [4]:
def Permut(A, i, j):
    P = np.copy(A)
    l = np.copy(P[i])
    P[i] = P[j]
    P[j] = l
    return P

In [5]:
def PLU(A, verbose=False):
    n = A.shape[0]
    Ak = np.copy(A)
    Lk = np.zeros((n,n))
    Pk = np.eye(n)
    for k in range(n):
        if verbose:
            print("Iteration",k)
        m = np.argmax(np.abs(Ak[k:,k]))+k
        Pk = Permut(Pk,k,m)
        Lkp12 = Permut(Lk,k,m)
        Akp12 = Permut(Ak,k,m)
        lk = np.zeros(n)
        if abs(Akp12[k,k])<1e-15:
            raise Exception("Le pivot est nul")
        lk[k:] = Akp12[k:,k]/Akp12[k,k]
        if verbose:
            print("lk=",lk)
        Lk = Lkp12
        Lk[k:,k] = lk[k:]
        if verbose:
            print("Lk=",Lk)
        Bk = np.zeros((n,n))
        Bk[k+1:,k:] = lk[k+1:].reshape((-1,1))*Akp12[k,k:].reshape((1,-1))
        if verbose:
            print("Bk=",Bk)
        Ak=Akp12-Bk
        if verbose:
            print("Ak=",Ak,"\n")
    P=Pk
    L=Lk
    M=Ak
    return P,L,M

In [6]:
A = np.array([[1,1,1,1],[1,1,3,3],[1,1,2,3],[1,3,3,3]])
P,L,U = PLU(A,verbose=True)
print("A=",A)
print("P=",P)
print("L=",L)
print("U=",U)
print("PLU=",P.transpose().dot(L.dot(U)))
if np.testing.assert_array_equal(L.dot(U),P.dot(A)) is None:
    print("PA=LU")

Iteration 0
lk= [1. 1. 1. 1.]
Lk= [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
Bk= [[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
Ak= [[1. 1. 1. 1.]
 [0. 0. 2. 2.]
 [0. 0. 1. 2.]
 [0. 2. 2. 2.]] 

Iteration 1
lk= [0. 1. 0. 0.]
Lk= [[1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
Bk= [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Ak= [[1. 1. 1. 1.]
 [0. 2. 2. 2.]
 [0. 0. 1. 2.]
 [0. 0. 2. 2.]] 

Iteration 2
lk= [0.  0.  1.  0.5]
Lk= [[1.  0.  0.  0. ]
 [1.  1.  0.  0. ]
 [1.  0.  1.  0. ]
 [1.  0.  0.5 0. ]]
Bk= [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 1.]]
Ak= [[1. 1. 1. 1.]
 [0. 2. 2. 2.]
 [0. 0. 2. 2.]
 [0. 0. 0. 1.]] 

Iteration 3
lk= [0. 0. 0. 1.]
Lk= [[1.  0.  0.  0. ]
 [1.  1.  0.  0. ]
 [1.  0.  1.  0. ]
 [1.  0.  0.5 1. ]]
Bk= [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Ak= [[1. 1. 1. 1.]
 [0. 2. 2. 2.]
 [0. 0. 2. 2.]
 [0. 0. 0. 1.]] 

A= [[1 1 1 1]
 [1 1 3 3]
 [1 1 2 3]
 [1 3 3 3]]
P= [[1. 

## 3) Résolution de systèmes triangulaires et comparaison des méthodes

Nous allons à présent comparer les 2 méthodes que nous venons d'implémenter. Le but est de tester ces méthodes sur un système linéaire. Or, on sait que pour résoudre un tel système il faut passer par la résolution de systèmes triangulaires (supérieurs et inférieurs). Ainsi, avant de passer à la suite, on se propose d'implémenter une méthode permettant de résoudre un système triangulaire, qu'il soit inférieur ou supérieur.

On rappelle que l'algorithme de résolution pour un système triangulaire de la forme

$$
Tx = b,
$$

s'écrit, si $T$ est triangulaire inférieure,

$$
x_0 = b_0/T_{0,0}\\
\textrm{Pour}\; k = 1,2,3,\cdots, n-1\; :
x _k = \left(b_k - \sum_{j = 0}^{k-1} T_{k,j} x_j\right)/T_{k,k},
$$

et, si $T$ est triangulaire supérieure,

$$
x_{n-1} = b_{n-1}/T_{n-1,n-1}\\
\textrm{Pour}\; k = n-2,\cdots, 0\; :
x _k = \left(b_k - \sum_{j = k+1}^{n-1} T_{k,j} x_j\right)/T_{k,k}.
$$

> **A faire :** Implémenter la fonction `Solve_Triang` qui permet de résoudre un système linéaire triangulaire (inférieur ou supérieur). Cette fonction prendra en entrée une matrice et un vecteur (correspondant aux données du système linéaire) ainsi qu'une variable précisant si le système est triangulaire supérieur ou inférieur et donnera en sortie un vecteur qui est la solution que l'on cherche.

In [7]:
def Solve_Triang(T, b, s=False):
    n = T.shape[1]
    x = np.zeros(n)
    if s:
        x[-1] = b[-1]/T[-1,-1]
        for k in range(n-2,-1,-1):
            x[k] = (b[k]-np.sum(T[k,k+1:].dot(x[k+1:])))/T[k,k]
    else:
        x[0] = b[0]/T[0,0]
        for k in range(1,n):
            x[k] = (b[k]-np.sum(T[k,:k].dot(x[:k])))/T[k,k]
    return x

On a charger, en début de ce notebook, la matrice $M$ de taille 100. A l'aide des deux méthodes de factorisation et de la méthode de résolution d'un système triangulaire, vous allez résoudre le système linéaire

$$
Mx = b,
$$

où $b\in\mathbb{R}^{100}$ est tel que $b_i = 1$, pour tout $0\leq i\leq n-1$.

> **A faire :** Résoudre le système précédent à l'aide de la factorisation LU standard et celle avec permutations. Comparer les résultats obtenus avec la solution obtenue avec `npl.solve(M,b)`. Quelle méthode donne le meilleur résultat?

In [8]:
M = np.load('TP6_Matrice.npy')
n = M.shape[0]
b = np.ones(n)
x_true = npl.solve(M,b)

print("PLU")
P,L,U = PLU(M)
y = Solve_Triang(L,P.dot(b),s=False)
x = Solve_Triang(U,y,s=True)
print(np.linalg.norm(x-x_true),"\n")

print("LU")
L,U = LU(M)
y = Solve_Triang(L,b,s=False)
x = Solve_Triang(U,y,s=True)
print(np.linalg.norm(x-x_true),"\n")

PLU
5.942172898183176e-16 

LU
1.0563353172803138e-12 

